In [ ]:
import pandas as pd
import os
import shutil

In [ ]:
train_csv_path = '/content/drive/MyDrive/Malaria/malaria/train_data.csv'
test_csv_path = '/content/drive/MyDrive/Malaria/malaria/test_data.csv'

In [ ]:
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

In [ ]:
combined_df = pd.concat([train_df, test_df])

In [ ]:
combined_df.sample(10)

,ImagePAth,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center,img_width,img_height,channels,pixels
71936,images/61575482-91e1-4db9-83c7-687b7ddbbfc4.png,red blood cell,1007.0,421.0,1091.0,505.0,84.0,84.0,1049.0,463.0,1600,1200,3,5760000
33404,images/f632981c-1330-4078-8a12-691beb2dab02.png,red blood cell,1392.0,336.0,1493.0,443.0,101.0,107.0,1442.5,389.5,1600,1200,3,5760000
5026,images/b7d0ae11-a393-4aec-8f68-6ba07147f78a.png,red blood cell,1465.0,113.0,1579.0,233.0,114.0,120.0,1522.0,173.0,1600,1200,3,5760000
62715,images/9e1b0217-0926-4fd1-bb9e-a815c05deb59.png,red blood cell,525.0,443.0,639.0,539.0,114.0,96.0,582.0,491.0,1600,1200,3,5760000
77529,images/2d2e81f3-e329-4765-9ba1-2c95f12dc9f3.png,red blood cell,392.0,816.0,494.0,918.0,102.0,102.0,443.0,867.0,1600,1200,3,5760000
41554,images/0fcd1ebe-ee4a-40a0-b787-29cee18bfa1b.png,red blood cell,942.0,172.0,1051.0,260.0,109.0,88.0,996.5,216.0,1600,1200,3,5760000
121,images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,red blood cell,983.0,584.0,1093.0,683.0,110.0,99.0,1038.0,633.5,1600,1200,3,5760000
14415,images/3b13e20f-ec0d-44ec-b2f4-eaebfa09150e.png,red blood cell,1143.0,127.0,1252.0,242.0,109.0,115.0,1197.5,184.5,1600,1200,3,5760000
56012,images/177a3141-fa93-426a-9b52-7da71547ce44.png,red blood cell,882.0,537.0,981.0,660.0,99.0,123.0,931.5,598.5,1600,1200,3,5760000
45162,images/2596eb32-1323-41e9-8c0f-64819abcfc73.png,red blood cell,71.0,188.0,186.0,300.0,115.0,112.0,128.5,244.0,1600,1200,3,5760000


In [ ]:
combined_df.shape

(86035, 14)

In [ ]:
combined_df['w_yolo'] = combined_df[['w', 'img_width']].apply(lambda x: x[0] / x[1], axis=1)
combined_df['h_yolo'] = combined_df[['h', 'img_height']].apply(lambda x: x[0] / x[1], axis=1)
combined_df['x_center_yolo'] = combined_df[['x_center', 'img_width']].apply(lambda x: x[0] / x[1], axis=1)
combined_df['y_center_yolo'] = combined_df[['y_center', 'img_height']].apply(lambda x: x[0] / x[1], axis=1)

<ipython-input-59-50fe3605dc5c>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_df['w_yolo'] = combined_df[['w', 'img_width']].apply(lambda x: x[0] / x[1], axis=1)
<ipython-input-59-50fe3605dc5c>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_df['h_yolo'] = combined_df[['h', 'img_height']].apply(lambda x: x[0] / x[1], axis=1)
<ipython-input-59-50fe3605dc5c>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combin

In [ ]:
yolov5_df = combined_df.copy()
yolov5_df = yolov5_df[yolov5_df["ImagePAth"].apply(lambda x: x[-4:] == '.png')].reset_index(drop=True)
yolov5_df["name"] = yolov5_df["ImagePAth"].apply(lambda x: x.split("/")[-1].split(".")[0])
yolov5_df.head()

,ImagePAth,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center,img_width,img_height,channels,pixels,w_yolo,h_yolo,x_center_yolo,y_center_yolo,name
0,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1440.0,1057.0,1540.0,1158.0,100.0,101.0,1490.0,1107.5,1600,1200,3,5760000,0.06250,0.084167,0.931250,0.922917,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55
1,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1303.0,868.0,1403.0,971.0,100.0,103.0,1353.0,919.5,1600,1200,3,5760000,0.06250,0.085833,0.845625,0.766250,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55
2,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,900.0,578.0,1008.0,689.0,108.0,111.0,954.0,633.5,1600,1200,3,5760000,0.06750,0.092500,0.596250,0.527917,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55
3,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,611.0,304.0,713.0,408.0,102.0,104.0,662.0,356.0,1600,1200,3,5760000,0.06375,0.086667,0.413750,0.296667,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55
4,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,881.0,198.0,1003.0,312.0,122.0,114.0,942.0,255.0,1600,1200,3,5760000,0.07625,0.095000,0.588750,0.212500,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55


In [ ]:
yolov5_df.category.unique()

array(['red blood cell', 'trophozoite', 'schizont', 'difficult', 'ring',
       'leukocyte', 'gametocyte'], dtype=object)

In [ ]:

class_mapping = {'red blood cell': 0,
                 'trophozoite':0,
                 'schizont':0,
                 'difficult':0,
                 'ring':0,
                 'leukocyte':0,
                 'gametocyte':0}

os.makedirs('yolov7/labels', exist_ok=True)

grouped = yolov5_df.groupby('ImagePAth')

for image_path, group in grouped:
    annotations = []
    for _, row in group.iterrows():
        class_id = class_mapping[row['category']]
        x_center = row['x_center_yolo']
        y_center = row['y_center_yolo']
        width = row['w_yolo']
        height = row['h_yolo']
        annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    image_name = os.path.splitext(os.path.basename(image_path))[0]
    label_file = os.path.join('yolov7/labels', f"{image_name}.txt")
    with open(label_file, 'w') as f:
        f.write("\n".join(annotations))


In [ ]:
!mv /content/yolov7 /content/drive/MyDrive/Malaria